In [10]:
import numpy as np
import scipy
import matplotlib.pyplot as plt 
from numpy.linalg import eig
import matplotlib.animation as animation
from IPython.display import clear_output

# General Parameters
mass_electron = 9.109534e-31 # 1.0 # [kg]
hbar = 6.582119569e-16 # [eV s] source: wikipedia
# length = 100.0 # [??]
number_steps = 500
electric_field = 0.0# 5 #0.01 # Electric field [V/nm]
electron_charge = 1 # 1.60217663e-19 # [C]
h_c = 1240 # eV nm

In [11]:
# plotting
PLOT_LIMIT = []#400,800]
Y_LIMIT = [] # leave blank for auto
ERROR_BARS = False
PLOT_FITTED = False
split_ = ","
LABEL_FONT_SIZE = 13
TICK_FONT_SIZE = 11
LINE_WIDTH = 0.5
LEGEND = True
label_x = ""
label_y = ""
plot_title = ""
aspect_ratio = [7,7]

colours = [ 'black','dimgrey','lightslategrey' ,'silver', 'cadetblue', 'teal','darkolivegreen','green','turquoise', 'olivedrab','seagreen']

def plot_graph(x, y): #create a single plot
    labels = []
    plt.figure()
    plt.rcParams["figure.figsize"] = (aspect_ratio[0],aspect_ratio[1])
    fig, ax = plt.subplots()
    plt.title(f"")
    #plt.xlabel(label_x, fontsize=LABEL_FONT_SIZE)
    #plt.ylabel(label_y,  fontsize=LABEL_FONT_SIZE)
    plt.xticks(fontsize = TICK_FONT_SIZE)
    plt.yticks(fontsize = TICK_FONT_SIZE)
    if(bool(Y_LIMIT) == True):
        plt.ylim(Y_LIMIT)
    if(bool(PLOT_LIMIT) == True):
        plt.xlim(PLOT_LIMIT)
    
    right_side = ax.spines["right"]
    top_side = ax.spines["top"]
    right_side.set_visible(False)
    top_side.set_visible(False)
    plt.plot(x, y, linewidth = LINE_WIDTH, color = 'dimgrey', marker = 's', markersize = 0.5, markerfacecolor='dimgrey')
    plt.grid(True, alpha=0.2)
    #labels = np.array(labels)
    #plt.savefig(f'{file}_figure.png', dpi = 1000, bbox_inches='tight')
    plt.show()

def plot_graphs(x, y): #create a single plot
    labels = []
    plt.figure()
    plt.rcParams["figure.figsize"] = (6,6)
    fig, ax = plt.subplots()
    plt.title(plot_title)
    plt.xlabel(label_x, fontsize=LABEL_FONT_SIZE)
    plt.ylabel(label_y,  fontsize=LABEL_FONT_SIZE)
    #ax.xaxis.set_minor_locator(AutoMinorLocator())
    #plt.xticks(fontsize = TICK_FONT_SIZE)
    #plt.yticks(fontsize = TICK_FONT_SIZE)
    if(bool(Y_LIMIT) == True):
        plt.ylim(Y_LIMIT)
    if(bool(PLOT_LIMIT) == True):
        plt.xlim(PLOT_LIMIT)
    
    right_side = ax.spines["right"]
    top_side = ax.spines["top"]
    right_side.set_visible(False)
    top_side.set_visible(False)
    i=0
    for ys in y:
        if i < len(colours):
            plt.plot(x, ys, linewidth = LINE_WIDTH, marker = 's', markersize = 0.5, color = colours[i])
        else:
            plt.plot(x, ys, linewidth = LINE_WIDTH, marker = 's', markersize = 0.5)
        i+=1
    plt.grid(True, alpha=0.2)
    if(LEGEND==True):
        plt.legend(legend)
    #labels = np.array(labels)
    #plt.savefig(f'{file}_figure.png', dpi = 1000, bbox_inches='tight')
    plt.show()

def make_array(y):
    p = np.zeros(number_steps)
    for nr in range(0, number_steps):
        p[nr] = y
    return p

In [12]:
# MATERIALS AND HETEROSTRUCTURE CLASSES
class Material:
    def __init__(self, electron_affinity, band_gap, e_eff_mass, lh_eff_mass, hh_eff_mass):
        self.electron_affinity = electron_affinity
        self.band_gap = band_gap
        self.e_eff_mass = e_eff_mass
        self.lh_eff_mass = lh_eff_mass
        self.hh_eff_mass = hh_eff_mass

def CBO(material1, material2): # Conduction Band Offset [eV] between 2 materials
    return (material1.electron_affinity - material2.electron_affinity)

def VBO(material1, material2): # Valence Band Offset [eV] between 2 materials
    calc1 = material1.electron_affinity + material1.band_gap
    calc2 = material2.electron_affinity + material2.band_gap
    return (calc2 - calc1)

class Layer:
    def __init__(self, material, thickness): # thickness in [nm]
        self.material = material
        self.thickness = thickness
        
class Heterostructure: 
    layers = np.empty(0, dtype=Layer)
    conduction_potential = np.empty(0, dtype=float)
    valence_potential = np.empty(0, dtype=float)
    heterostructure_thickness = 0.0
    
    def __init__(self, layers): # thickness in [nm]
        self.layers = layers
        self.conduction_potential = np.array(0)
        self.valence_potential = np.array(0)
        total_thickness = 0
        for layer in self.layers:
            total_thickness += layer.thickness
        self.heterostructure_thickness = total_thickness
        #self.set_potentials()

    def add_layer(self, layer):
        self.layers.append(layer)
        #set_potentials()
        
    def clear_layers(self):
        self.layers.clear()
                
    def potential(self, particle, x): # potential as a function of x inside the structure (specific band energy)
        material_threshold = 0.0
        U = 0.0
        V = 0.0
        i = 0
        
        for layer in self.layers:
            material_threshold += layer.thickness
            if x >= material_threshold:
                if(over_ride_offsets == False):
                    U += CBO(self.layers[i].material, self.layers[i+1].material)
                    V += VBO(self.layers[i].material, self.layers[i+1].material)
                elif(over_ride_offsets == True):
                    U += CBO_override[i]
                    V += VBO_override[i]
                i = i + 1
            else: # if used correctly, there should never be an error here (one might occur when an x input is out of analysis bounds)
                if particle == 1 or particle == 2: # if particle is hole
                    return V - electron_charge*electric_field*(x-self.heterostructure_thickness) # not as efficient as defining a strict function globally but gives ALOT of freedom (only adds +(number of layers)*3 computations at most  
                elif particle == 0:                # else it is electron
                    return U + electron_charge*electric_field*x
                
    def eff_mass(self, particle, x): # get effective mass for material at x in semiconductor wafer
        material_threshold = 0.0
        for layer in self.layers:
            material_threshold += layer.thickness
            if x <= material_threshold:
                if particle == 'hh':
                    return layer.material.hh_eff_mass
                if particle == 'lh':
                    return layer.material.lh_eff_mass
                if particle == 'e':
                    return layer.material.e_eff_mass

In [13]:
# Example Materials // BG = Bandgap, EF = Electron Affinity @ 300K
BG_GaAs = 1.424 # [eV]
EF_GaAs = 4.07 # [eV] @ 300K !
x_ratio = 0.3
EF_AlGaAs = 4.07 - 1.1*x_ratio # [eV] (x<0.45) @ 300K !
BG_AlGaAs = 1.424 + 1.247*x_ratio # [eV] (x<0.45)

# Decleration: Material(EF, BG, e_eff_mass, lh_eff_mass, hh_eff_mass) @ 300K
GaAs = Material(EF_GaAs, BG_GaAs, 0.063*mass_electron, 0.082*mass_electron, 0.51*mass_electron)
AlGaAs = Material(EF_AlGaAs, BG_AlGaAs, (0.063+x_ratio*0.083)*mass_electron, (0.082+x_ratio*0.068)*mass_electron, (0.51+x_ratio*0.25)*mass_electron)
Free_Space = Material(0.0, 0.0, mass_electron, mass_electron, mass_electron)
GaInAs = Material(4.5, (0.36+0.63*x_ratio+0.43*x_ratio**2), 0.041*mass_electron, 0.052*mass_electron, 0.45*mass_electron)
InP = Material(4.38, 1.344, 0.08*mass_electron, 0.089*mass_electron, 0.6*mass_electron)

In [14]:
potential_in_use = []
def solve(heterostructure): # Arbitrary, one operation, get all possible states/wavefunctions for a given structure
    Energies = np.zeros((3,number_steps))
    #eigenVector_ = [[0]*number_steps]*3
    eigenVector_ = [[0]*number_steps for _ in range(3)]

    for p in range(0, 3): # solve for electron, heavy hole, and light hole
        M = np.zeros((number_steps, number_steps))
        x = np.zeros(number_steps)

        delta_x = heterostructure.heterostructure_thickness / (number_steps + 1.0)
        x[0] = delta_x

        alpha_w = np.zeros(number_steps)
        alpha_e = np.zeros(number_steps)
        alpha_x = np.zeros(number_steps)

        particle = ['e', 'lh', 'hh']

        alpha_w[0] = -(hbar*hbar/2.0) * 1.0/(delta_x*delta_x) * 2.0/(heterostructure.eff_mass(particle[p],x[0])+heterostructure.eff_mass(particle[p],x[0]))
        alpha_e[0] = -(hbar*hbar/2.0) * 1.0/(delta_x*delta_x) * 2.0/(heterostructure.eff_mass(particle[p],x[0])+heterostructure.eff_mass(particle[p],x[1]))
        alpha_x[0] = -alpha_w[0]-alpha_e[0]

        M[0][0] = alpha_x[0] + potential_in_use[p][0]
        M[0][1] = alpha_e[0]

        for nr in range(0, number_steps):
            x[nr] = x[0] + nr*delta_x

        for nr in range(1, number_steps-1): #range runs from 1 to num_steps-2 (stupid)
            alpha_w[nr] = -(hbar*hbar/2.0) * 1.0/(delta_x*delta_x) * 2.0/(heterostructure.eff_mass(particle[p],x[nr])+heterostructure.eff_mass(particle[p],x[nr-1]))
            alpha_e[nr] = -(hbar*hbar/2.0) * 1.0/(delta_x*delta_x) * 2.0/(heterostructure.eff_mass(particle[p],x[nr])+heterostructure.eff_mass(particle[p],x[nr+1]))
            alpha_x[nr] = -alpha_w[nr]-alpha_e[nr]

            M[nr][nr-1] = alpha_w[nr]    #sub-diagonal
            M[nr][nr] = alpha_x[nr] + potential_in_use[p][nr] #diagonal
            M[nr][nr+1] = alpha_e[nr]   #upper diagonal 

        alpha_w[number_steps-1] = -(hbar*hbar/2.0) * 1.0/(delta_x*delta_x) * 2.0/(heterostructure.eff_mass(particle[p],x[number_steps-1])+heterostructure.eff_mass(particle[p],x[number_steps-1-1]))
        alpha_e[number_steps-1] = -(hbar*hbar/2.0) * 1.0/(delta_x*delta_x) * 2.0/(heterostructure.eff_mass(particle[p],x[number_steps-1])+heterostructure.eff_mass(particle[p],x[number_steps-1])) # assuming m(x_edge-dx) = m(x_edge) as boundary condition
        alpha_x[number_steps-1] = -alpha_w[number_steps-1]-alpha_e[number_steps-1]
        M[number_steps-1][number_steps-2] = alpha_w[number_steps-1]
        M[number_steps-1][number_steps-1] = alpha_x[number_steps-1] + potential_in_use[p][number_steps-1]

        w,v=eig(M)

        idx = np.argsort(w) # sort in terms of energies
        w = w[idx]
        v = v[:,idx]
        
        for i in range(number_steps):
            #print(len(eigenVectors[0]))
            Energies[p][i] = w[i]
            eigenVector_[p][i] = v[:,i]
        
    return x, Energies, eigenVector_

def relative_energy(energy, p): # accept a 2 paramters [energy relative to well of..][particle type]
    # corrects an (inital) calculated energy from QW solution for respective band of type particle type p
    EF_offset = electron_charge*electric_field*QW.heterostructure_thickness
    BG = min(abs(layers[i].material.band_gap) for i in range (0, len(layers)-1)) # this should be okay with and without override on    
    if(p==0):
        if over_ride_offsets == True:
            E_REL = energy + BG + max(abs(CBO_override[i]) for i in range(0, len(layers)-1))
        else:
            E_REL = energy + BG + max(abs(CBO(layers[i].material, layers[i+1].material)) for i in range(0, len(layers)-1))
    else: #(p==1 or p==2):
        if over_ride_offsets == True:
            E_REL = -max(abs(VBO_override[i]) for i in range(0, len(layers)-1)) - energy + EF_offset
        else:
            E_REL = -max(abs(VBO(layers[i].material, layers[i+1].material)) for i in range(0, len(layers)-1)) - energy + EF_offset
    return E_REL

def find_transitions(energies): # accept a SORTED 2 Dimensional matrix [3][number_solutions] ... 
    E_gap = np.zeros((number_steps, number_steps, 2))
    for k in range(2): # for 2 hole types
        for i in range(len(energies[0])): # for all electrons
            for j in range(len(energies[0])): # and all hole states
                E_gap[i][j][k] = abs(energies[0][i]-energies[k+1][j])
    return E_gap # a matrix/array of energy transtions indexed as [electron state][hole state][hole type] 

def find_energies_relative(energies): # accept a 2 Dimensional matrix [3][number_solutions]
    energies_relative = np.zeros((3, number_steps))
    for k in range(3): # for 3 particle types
        for i in range(len(energies[0])): # for all states
            energies_relative[k][i] = relative_energy(energies[k][i], k)
    return energies_relative # returns [3][number_solutions] matrix of energies relative to their ectual value in well structure

def overlap_integral(vector1, vector2):    # S = <v_1|v_2> | accept only vectors of same length
    overlap = np.zeros(len(vector1))       # assuming both are the same length
    N1 = sum(abs(vector1)**2)              # Normalisation constant N = <v|v>
    N2 = sum(abs(vector2)**2)
    #print("N1 = {}".format(N1))
    #print("N2 = {}".format(N2))
    vector1_dummy = vector1 / N1           # normalise wavefunctions (N^-1)|v>
    vector2_dummy = vector2 / N2
    for i in range(len(vector1)):
        overlap[i] = (vector1_dummy[i]*vector2_dummy[i]) 
    return overlap                         # I = <v_1|v_2>

def I_squared(vector1, vector2):           # S = <v_1|v_2> | accept only vectors of same length
    overlap = overlap_integral(vector1, vector2)
    I_squared = sum(abs(overlap))**2
    return I_squared            # I^2 = |<v_1|v_2>|^2

def find_overlaps_all(wavefunctions): # accept matrix of size [particle type (0,1,2)][number_steps] directly from solve(QW)
    I_squared_matrix = np.zeros((number_steps, number_steps, 2)) # [electron state][hole state][hole type]
    for k in range(2): # for 2 hole types
        for i in range(len(wavefunctions[0])): # for all electrons
            state1 = wavefunctions[0][i]
            for j in range(len(wavefunctions[0])): # and all hole states
                state2 = wavefunctions[1+k][j] # [1+k] so that we index correct state - wavefunctions indexed as [particle type (0,1,2)][number_steps]
                I_squared_matrix[i][j][k] = I_squared(state1, state2)
    return I_squared_matrix # a matrix/array of wavefunction overlaps indexed as [electron state][hole state][hole type]

def convert_to_wavelength(E_GAP): # Gap between [electron state] and [hole state] of [hole type]
    E_GAP_WL = np.zeros((number_steps, number_steps, 2))
    for k in range(2): # for 2 hole types
        for i in range(number_steps): # for all electrons
            for j in range(number_steps): # and all hole states
                E_GAP_WL[i][j][k] = h_c / E_GAP[i][j][k]
    return E_GAP_WL

In [15]:
# DEFINE STRUCTURE ---------------------------------------------------------------------------
layer1 = Layer(AlGaAs, 10) # 10nm thick layer of GaAs
layer2 = Layer(GaAs, 10) # 10nm thick layer of AlGaAs
layers = [layer1, layer2, layer1]
QW = Heterostructure(layers)

# OVERRIDE BAND OFFSETS | Done by hand : Array lengths must be num.layers-1 ------------------
electric_field = 0.01
over_ride_offsets = True
E_gap_diff = np.abs(-layer2.material.band_gap + layer1.material.band_gap)
conduction_band_offset_ratio = 0.6
valence_band_offset_ratio = 0.4
print(E_gap_diff)

# VBO_override = [-layer2.material.band_gap*valence_band_offset_ratio, layer2.material.band_gap*valence_band_offset_ratio]
# CBO_override = [-layer2.material.band_gap*conduction_band_offset_ratio, layer2.material.band_gap*conduction_band_offset_ratio]

VBO_override = [-E_gap_diff*valence_band_offset_ratio, E_gap_diff*valence_band_offset_ratio]
CBO_override = [-E_gap_diff*conduction_band_offset_ratio, E_gap_diff*conduction_band_offset_ratio]
print(VBO_override)
print(CBO_override)

0.3741000000000001
[-0.14964000000000005, 0.14964000000000005]
[-0.22446000000000005, 0.22446000000000005]


In [17]:
# QWI / set potential ----------------------------------------------------------------------------------------
x, energies, wavefunctions = solve(QW) # returns 2D matrices, index [particle type][solution in order from lowest]
electric_field = 0
# Convolution of well potential with gaussian
pot_org = np.zeros(number_steps)
pot_QWI = np.zeros((3,number_steps))
func_x = np.zeros(number_steps)
Gauss_y = np.zeros(number_steps)
x_0 = number_steps / 2
sigma = 30
for p in range(3):
    for i in range(number_steps):
        pot_org[i] = QW.potential(p,x[i])
        func_x[i] = i
        Gauss_y[i] = 1.0/np.sqrt(np.sqrt(2*np.pi)*sigma)*np.exp(-(func_x[i]-x_0)*(func_x[i]-x_0)/(2.0*sigma*sigma))

    N_G = abs(sum(Gauss_y))
    Gauss_y = Gauss_y/N_G
    
    aspect_ratio=[3,3]
    plot_graph(x, Gauss_y)   

    pot_org_new = pad_func_zeros(pot_org)
    Gauss_y_new = pad_func_zeros(Gauss_y)
    x_new = pad_func_linear(x)

    LEGEND=False
    pot_QWI[p] = np.roll(convolve(pot_org_new, Gauss_y_new), int(number_steps))[int(number_steps*2/4):int(number_steps*2*3/4)]
    aspect_ratio=[5,5]
    plot_graphs(x, [pot_QWI[p], pot_org])    

    print("Potentials have the same 'area'")
    print(sum(pot_org))
    print(sum(pot_QWI[p]))

#potential_in_use = pot_org

#         or

potential_in_use = pot_QWI

# SOLVE --------------------------------------------------------------------------------------
x, energies, wavefunctions = solve(QW) # returns 2D matrices, index [particle type][solution in order from lowest]
# RESULTS ------------------------------------------------------------------------------------

relative_energies = find_energies_relative(energies)
E_GAP = find_transitions(relative_energies)

PLOT_LIMIT=[]

plot_graphs(x, [wavefunctions[0][0], wavefunctions[1][0]])
plot_graph(x,wavefunctions[0][0])
plot_graph(x,wavefunctions[1][0])

IndexError: list index out of range